### Imports

In [ ]:
%pip install openai --upgrade
%pip install pandas

In [ ]:
import os
import csv
import openai
import pandas as pd

### OPENAI API Keys

In [ ]:
openai.api_key = 'PLEASE_GET_YOUR_KEY_AND_MAKE_SURE_YOU_HAVE_SAVINGS_IN_OPENAI'

### Loading Dataset

In [84]:
df = pd.read_csv("the_predict_result", sep='\t', quoting=csv.QUOTE_NONE)
df = df[:100]

In [85]:
ids = df.id.values
label = df.label.values

In [86]:
textdf = pd.read_csv("validate.csv", delimiter=',', index_col=False, encoding='latin1')
textdf = textdf[:100]
text = textdf.text.values
truelabel = textdf.label.values

In [ ]:
# Compare the performance
print(classification_report(truelabel, label, target_names=['negative', 'positive']))

In [ ]:
def find_longest_string_length(lst):
    longest_str = max(lst, key=len)
    return len(longest_str)

result = find_longest_string_length(text)
print("The longest string length of this dataset is:", result)

In [ ]:
def count_words_in_longest_string(lst):
    longest_str = max(lst, key=len)
    words_in_longest_str = len(longest_str.split())
    return words_in_longest_str

result = count_words_in_longest_string(text)
print("The longest text length is:", result)


In [ ]:
def average_word_count(lst):
    total_words = 0
    total_strings = len(lst)
    for string in lst:
        words_in_string = len(string.split())
        total_words += words_in_string

    average_words = total_words / total_strings
    return average_words

result = average_word_count(text)
print("The average word number is:", result)

In [26]:
prompts = []
for index, value in enumerate(label):
    input = text[index] + ' Label:' + str(value)
    prompts.append(input)

In [27]:
l = label
t = text

#### Constraint

In [29]:
prompts = []
id1 = []
true = []
for i, v in enumerate(t):
    if l[i] == 1:
        prompts.append(v)
        id1.append(ids[i])
        true.append(truelabel[i])
        

### Prompt

In [31]:
prompt = '''
You are given a text with a label in the end, your job is to take the label in the end and identify whether it is true or false according to the following rules:
If the label is 0, please return True directly.
If the label is 1 and it is True,  then the text should present one of the following cases:
1. Users explicitly report having a diagnosis of SAD.
Examples: “I've been diagnosed with SA”, “My therapists diagnosed me with social phobia”, “When I was 15 I was diagnosed with SAD”.
2. Users report having a diagnosis but the disease is not mentioned (in those cases SAD is assumed).
Examples: “Yes, I was diagnosed 2 months ago”, “I have a diagnosis”, “My therapists confirmed I have it”.
3. Users report their doctor (therapist/psychologist/etc) agrees or believes they have SAD.
Examples: “My therapist agreed with my self-diagnosis”.
4. Users report having a therapist or attending therapy (now or in the past) for SAD. The disorder can be explicitly stated or it can be implied.
Examples: “I've only gone to a few sessions so far”, “I talked with my therapists about [...]”, “I was on therapy for a long time”.
5. Users report going to a doctor or talking to a doctor about SAD.
Examples: “I eventually found a doctor I felt a good connection with so I felt comfortable sharing.”
6. Users report taking medication for SAD (now or in the past).
Examples: “I started taking zoloft last year and my anxiety is so much better”, “Therapy nor meds worked for me”.

If the label is 1 and it is False, then the text should present one of the following cases:
1. Users report not having a diagnosis of SAD. Examples: “I've never been diagnosed”
2. Users report having a diagnosis of another disease/disorder but no mention of SAD. Example: “I've been diagnosed with depression”.
3. Users report going to a counselor or school counselor. Example: “I go to my school counselor regularly.”
4. Users report being self-diagnosed. Examples: “I'm self-diagnosed”, “I did self diagnose with SA ”
5. Users report doing a specific therapy with no professional intervention.
Examples: “I've been doing exposure therapy by myself”, “I've been doing self counseling”.
6. Users report taking medication with no prescription.
7. Users report going to a doctor but not necessarily for SAD.
Example: “I only go out for necessary interactions like going to the doctor”, “My mom used to make me go to counseling after my parents got a divorce”
8. The user reports having social anxiety but makes no mention of a diagnosis or treatment.
Example: “I was just recently diagnosed with ADHD-PI. [...] The combination of SA and ADHD sucks. Even when I really want to, I find it hard to focus on what people are saying. I get bits and pieces of what people are saying and it's frustrating and makes my SA worse. ”, “”
9. All other cases

Here are some examples: 
Input: I've been diagnosed with SA, Label: 0
Output: False, because the user explicitly said he has SA. But the given label is 0.
Input My therapists diagnosed me with social phobia. Label 1
Output: True, because the user explicitly said he has SA.

Your output should only be True or False with reasoning sentences that can show the proof step by step within 30 words. 
You should return True for most uncertain cases and be careful to return False.
'''
print(prompt)

In [33]:
def prepare_message(uinput):
    #preparing the prompt for OpenAI 
    role="user"
    #prompt="Where is Tahiti located?" #maintenance or if you do not want to use a microphone
    line = {"role": role, "content": uinput}
    #creating the mesage
    assert1={"role": "system", "content": "You are a highly intelligent and accurate social anxiety explainable system."}
    assert2={"role": "assistant", "content": prompt}
    assert3=line
    iprompt = []
    iprompt.append(assert1)
    iprompt.append(assert2)
    iprompt.append(assert3)

    return iprompt

### GPT3.5 inference pipeline

In [34]:
def gpt(uinput):
    iprompt=prepare_message(uinput) #preparing the messages for ChatGPT
    response=openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=iprompt) #ChatGPT dialog
    text=response["choices"][0]["message"]["content"] #response in JSON
    print("ChatGPT response:",text)
    return text

In [37]:
import time
predicts = []
for t, i in enumerate(prompts):
    if (t + 1) % 3 == 0:
        time.sleep(60)  # 停顿一分钟
    
    print(t+ 1,':', id1[t])
    inputs = i
    predict = str(gpt(inputs))
    predicts.append(predict)

1 : 79
ChatGPT response: True. The user mentions feeling awkward and hating reconnecting with people, being uncomfortable eating in front of others, experiencing difficulty ordering food, and feeling awkward at parties. These are all indicative of social anxiety symptoms.
2 : 81
ChatGPT response: True. The user discusses their own experience with social anxiety and how they have managed it without therapy by creating small goals for themselves. They mention severe social anxiety, trouble leaving the house, and having a job and new job opportunities. They provide encouraging advice and suggest starting with small goals and believing in oneself.
3 : 185
ChatGPT response: True. The user explicitly mentioned going to therapy for social anxiety and the positive impact it is having.
4 : 204
ChatGPT response: True. The user explicitly states being diagnosed with social anxiety and expresses a desire for a good relationship, highlighting the impact of social anxiety on their intimate emotions 

### Correction

In [57]:
correction = []
for i , v in enumerate(predicts):
    if "false" in v or "False" in v or "FALSE" in v:
        correction.append(id1[i])
        

In [75]:
pre = label.copy()

In [77]:
for i , v in enumerate(ids):
    if v in correction:
        print('correct id', v)
        label[i] = 0

correct id 522
correct id 586
correct id 1047
correct id 1534
correct id 1556
correct id 1577
correct id 1924
correct id 2012
correct id 2529


In [ ]:
# Compare the performance
print(classification_report(truelabel, pre, target_names=['negative', 'positive']))

#### Save the resulte

In [90]:
def write_list_to_csv(ids_list, data_list, file_path):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['ids', 'correction', 'explanation'])
        for ids, item in zip(ids_list, data_list):
            words = item.split()
            correction = words[0]
            explanation = ' '.join(words[1:])
            writer.writerow([ids, correction, explanation])

output_file_path = 'ALEX_task4_label1_corrected.csv'
write_list_to_csv(id1, predicts, output_file_path)